In [1]:
from scipy import stats

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

import seaborn as sns

from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import os

import graphviz
from graphviz import Graph

import env
import acquire_telco
import prepare_telco

import warnings
warnings.filterwarnings("ignore")

![](telco_churn_pic.png)

### PLAN

In [2]:
# project plan and goals in readme

In [ ]:
# work through pipeline

In [ ]:
# data dictionary 

In [3]:
# Clearly stating hypotheses(and add the testing of these to your task list)

#### Hypothesis testing
α = .05
#### hypothesis 1
$H_O$ - there is no affect on weather or not a customer has autopayment and churns

$H_A$ - There is an affect on churn, depending on weather or not a customer has autopay
#### hypothesis 2
$H_O$ - The number of additional services will not have an affect on churn. (refering to addons column)

$H_A$ - The number of additional services will have an affect on churn. (referring to addons column)

## Acquire

- created a python function that will automatically call a sql query that joins all of the tables in telco churn
- added my get connection so that i can connect sql to python to create my pandas database
- set the function to cache my telco data to a csv so i dont need to connect to sql every time i acquire the telco database

In [8]:
df = acquire_telco.get_telco_data()

In [9]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,internet_service_type_id,internet_service_type,payment_type_id,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,1,DSL,4,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,1,DSL,3,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,1,DSL,4,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,1,DSL,3,Bank transfer (automatic)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

In [11]:
df.describe()

,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,contract_type_id,internet_service_type_id,payment_type_id
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,1.872923,1.690473,2.315633,64.761692,1.690473,1.872923,2.315633
std,0.368612,24.559481,0.737796,0.833755,1.148907,30.090047,0.833755,0.737796,1.148907
min,0.000000,0.000000,1.000000,1.000000,1.000000,18.250000,1.000000,1.000000,1.000000
25%,0.000000,9.000000,1.000000,1.000000,1.000000,35.500000,1.000000,1.000000,1.000000
50%,0.000000,29.000000,2.000000,1.000000,2.000000,70.350000,1.000000,2.000000,2.000000
75%,0.000000,55.000000,2.000000,2.000000,3.000000,89.850000,2.000000,2.000000,3.000000
max,1.000000,72.000000,3.000000,3.000000,4.000000,118.750000,3.000000,3.000000,4.000000


In [14]:
df.value_counts

<bound method DataFrame.value_counts of      customer_id  gender  senior_citizen partner dependents  tenure  \
0     0016-QLJIS  Female               0     Yes        Yes      65   
1     0017-DINOC    Male               0      No         No      54   
2     0019-GFNTW  Female               0      No         No      56   
3     0056-EPFBG    Male               0     Yes        Yes      20   
4     0078-XZMHT    Male               0     Yes         No      72   
...          ...     ...             ...     ...        ...     ...   
7038  9962-BFPDU  Female               0     Yes        Yes       1   
7039  9967-ATRFS  Female               0      No         No      19   
7040  9970-QBCDA  Female               0      No         No       6   
7041  9975-SKRNR    Male               0      No         No       1   
7042  9986-BONCE  Female               0      No         No       4   

     phone_service    multiple_lines internet_service_type_id  \
0              Yes               Yes      

In [15]:
df.isnull().sum()

customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
internet_service_type_id    0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
contract_type_id            0
paperless_billing           0
payment_type_id             0
monthly_charges             0
total_charges               0
churn                       0
contract_type_id            0
contract_type               0
internet_service_type_id    0
internet_service_type       0
payment_type_id             0
payment_type                0
dtype: int64

In [25]:
df.churn.value_counts()

No     5174
Yes    1869
Name: churn, dtype: int64

In [ ]:
# ~ 73.5% yes
# ~ 26.5% no

### Choosing what to plot
##### scatterplot
- Tenure, total_charges, monthly charges

## Prepare